In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from random import randint
import numpy as np
import pandas as pd 

In [5]:
class CircleNet(nn.Sequential): 
    def __init__(self, res, dropout, n_out): 
        super(CircleNet, self).__init__()
        self.n_in = res
        self.affine1 = nn.Linear(self.n_in, self.n_in//2)
        self.drop1 = nn.Dropout(dropout)
        self.affine2 = nn.Linear(self.n_in//2, self.n_in//2)
        self.drop2 == nn.Dropout(dropout)
        self.affine3 = nn.Linear(self.n_in//2, n_out)
    def forward(self, x): 
        x = F.relu(self.affine1(x))
        x = self.drop1(x)
        x = F.relu(self.affine2(x))
        x = self.drop2(x)
        x = F.relu(self.affine3(x))
        x = F.softmax(x, dim = 1)

In [24]:
## make some uniform circular data
res = 36
data = np.zeros([1000,37])
for i in range(1000): 
    for j in range(1000): 
        num = randint(0,35)
        data[i][num]+= 1
        data[i][36] = -2
np.savetxt("uniformtest2.csv", data, delimiter = ",")

In [11]:
##reads the data back into a numpy array
data2 = pd.read_csv("uniform.csv")
data2array = data2.values

array([32., 33., 23., 26., 28., 27., 25., 40., 26., 33., 20., 25., 25.,
       25., 31., 34., 32., 28., 26., 22., 23., 32., 28., 25., 37., 18.,
       33., 20., 23., 26., 36., 24., 26., 37., 20., 31.])

In [18]:
import os
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset
import torch

In [237]:
##This is a class that allows us to load data from csv files and make them into datasets that the dataloader can use
class CircularDataset(Dataset): 
    def __init__(self, data_root): 
        self.data = []
        self.dist_coder = LabelEncoder()
        self.dist_list = []
        for path in data_root: 
            array = pd.read_csv(path)
            
            self.dist_list += [array.values[0,-1]] ##get the distribution type for each file and add it to the list
            if len(self.data) == 0: 
                self.data = array.values
            else:
               
                self.data = np.concatenate((self.data,array.values), axis = 0)
                
               
       
        self.dist_coder.fit(self.dist_list)
      
    def __getitem__(self,idx): 
        return self.data[idx][0:-1], self.to_one_hot(self.dist_coder,([self.data[idx,-1]]))[0]
    def __len__(self): 
        return len(self.data)
    
    def to_one_hot(self, codec, values):
        value_idxs = codec.transform(values)
        return torch.eye(len(codec.classes_))[value_idxs]

In [238]:
[1,2,3][0:-1]

[1, 2]

In [236]:
dataset = CircularDataset(["uniformtest1.csv", "uniformtest2.csv"])

(999, 37)
(999, 37)
[[22. 28. 33. ... 33. 31. -1.]
 [25. 27. 33. ... 22. 31. -1.]
 [30. 27. 25. ... 22. 32. -1.]
 ...
 [26. 26. 27. ... 34. 26. -1.]
 [30. 32. 27. ... 37. 28. -1.]
 [15. 33. 32. ... 37. 21. -1.]]
[[31. 34. 30. ... 22. 31. -2.]
 [29. 29. 31. ... 32. 20. -2.]
 [23. 26. 29. ... 32. 27. -2.]
 ...
 [26. 28. 24. ... 18. 27. -2.]
 [22. 16. 30. ... 30. 40. -2.]
 [28. 29. 38. ... 19. 19. -2.]]
[[22. 28. 33. ... 33. 31. -1.]
 [25. 27. 33. ... 22. 31. -1.]
 [30. 27. 25. ... 22. 32. -1.]
 ...
 [26. 26. 27. ... 34. 26. -1.]
 [30. 32. 27. ... 37. 28. -1.]
 [15. 33. 32. ... 37. 21. -1.]]


In [190]:
dataset.__getitem__(0)

(array([22., 28., 33., 27., 22., 30., 28., 30., 25., 27., 27., 26., 24.,
        25., 29., 27., 30., 25., 25., 28., 28., 28., 25., 24., 32., 24.,
        31., 31., 40., 30., 27., 27., 23., 28., 33., 31.]), tensor([0., 1.]))

In [191]:
iterator = torch.utils.data.DataLoader(dataset, batch_size = 2, shuffle = True)

In [192]:
for data, label in iterator: 
    
    print(label)

tensor([[0., 1.],
        [0., 1.]])
tensor([[0., 1.],
        [0., 1.]])
tensor([[0., 1.],
        [0., 1.]])
tensor([[0., 1.],
        [0., 1.]])
tensor([[0., 1.],
        [0., 1.]])
tensor([[0., 1.],
        [0., 1.]])
tensor([[0., 1.],
        [0., 1.]])
tensor([[0., 1.],
        [0., 1.]])
tensor([[0., 1.],
        [0., 1.]])
tensor([[0., 1.],
        [0., 1.]])
tensor([[0., 1.],
        [0., 1.]])
tensor([[0., 1.],
        [0., 1.]])
tensor([[0., 1.],
        [0., 1.]])
tensor([[0., 1.],
        [0., 1.]])
tensor([[0., 1.],
        [0., 1.]])
tensor([[0., 1.],
        [0., 1.]])
tensor([[0., 1.],
        [0., 1.]])
tensor([[0., 1.],
        [0., 1.]])
tensor([[0., 1.],
        [0., 1.]])
tensor([[0., 1.],
        [0., 1.]])
tensor([[0., 1.],
        [0., 1.]])
tensor([[0., 1.],
        [0., 1.]])
tensor([[0., 1.],
        [0., 1.]])
tensor([[0., 1.],
        [0., 1.]])
tensor([[0., 1.],
        [0., 1.]])
tensor([[0., 1.],
        [0., 1.]])
tensor([[0., 1.],
        [0., 1.]])
t

        [0., 1.]])
tensor([[0., 1.],
        [0., 1.]])
tensor([[0., 1.],
        [0., 1.]])
tensor([[0., 1.],
        [0., 1.]])
tensor([[0., 1.],
        [0., 1.]])
tensor([[0., 1.],
        [0., 1.]])
tensor([[0., 1.],
        [0., 1.]])
tensor([[0., 1.],
        [0., 1.]])
tensor([[0., 1.],
        [0., 1.]])
tensor([[0., 1.],
        [0., 1.]])
tensor([[0., 1.],
        [0., 1.]])
tensor([[0., 1.],
        [0., 1.]])
tensor([[0., 1.],
        [0., 1.]])
tensor([[0., 1.],
        [0., 1.]])
tensor([[0., 1.],
        [0., 1.]])
tensor([[0., 1.],
        [0., 1.]])
tensor([[0., 1.],
        [0., 1.]])
tensor([[0., 1.],
        [0., 1.]])
tensor([[0., 1.],
        [0., 1.]])
tensor([[0., 1.],
        [0., 1.]])
tensor([[0., 1.],
        [0., 1.]])
tensor([[0., 1.],
        [0., 1.]])
tensor([[0., 1.],
        [0., 1.]])
tensor([[0., 1.],
        [0., 1.]])
tensor([[0., 1.],
        [0., 1.]])
tensor([[0., 1.],
        [0., 1.]])
tensor([[0., 1.],
        [0., 1.]])
tensor([[0., 1.],
 